In [80]:
import sys
sys.path.append("../src")
import os

import pandas as pd
import numpy as np
from etf_transformations import *
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.preprocessing import FunctionTransformer

In [81]:
df = pd.read_csv("../data/preprocessed/final_etf_data/weekends_aggregated_embedding/XLF_v4.csv")
df = sign_next_day(df)
df = drop_sign_and_return(df)


In [82]:
df.head(10)

,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,-0.041143,0.014036,-0.033065,-0.033968,0.009230,-0.046950,0.004274,0.024519,1.0,-1.0
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,-0.059143,0.007399,-0.055302,-0.039515,-0.000138,-0.066466,0.005075,-0.024416,1.0,-1.0
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,-0.056243,0.016681,-0.040932,-0.007561,0.011109,-0.076687,0.010291,-0.010929,1.0,-1.0
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,-0.033069,0.029601,-0.046962,-0.005147,0.038169,-0.097698,0.013953,-0.036602,1.0,1.0
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,-0.031487,0.017077,-0.037181,-0.028825,0.034319,-0.078715,0.038083,-0.010276,1.0,-1.0
5,2018-03-27,23.511696,0.208302,0.501280,0.290419,8.0,-0.082117,0.008858,-0.044920,-0.005257,...,-0.059449,0.005357,-0.036538,-0.046846,0.031692,-0.089109,0.010091,0.004574,1.0,1.0
6,2018-03-28,23.554998,0.092945,0.652902,0.254153,8.0,-0.161208,0.003700,-0.017640,0.008789,...,-0.044302,0.029991,-0.031216,-0.029270,0.003462,-0.093493,-0.006516,0.024834,1.0,1.0
7,2018-03-29,23.875416,0.055242,0.592329,0.352428,5.0,-0.297186,0.000013,-0.013250,-0.006634,...,-0.016003,0.012979,-0.054554,-0.006435,-0.023151,-0.084078,0.012592,0.008806,1.0,-1.0
8,2018-04-02,23.347151,0.046485,0.476633,0.476883,6.0,-0.430398,-0.013063,-0.034718,-0.003257,...,-0.014027,0.002121,-0.049837,0.019447,0.047689,-0.111716,0.016507,-0.010199,1.0,1.0
9,2018-04-03,23.667574,0.627429,0.324922,0.047649,3.0,0.579779,0.022106,-0.012358,0.013611,...,-0.024894,-0.010319,-0.062131,0.030860,0.067125,-0.091592,-0.007063,0.020872,1.0,1.0


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Columns: 393 entries, Date to Sign_next_day
dtypes: float64(392), object(1)
memory usage: 1.8+ MB


In [84]:
df.describe()

,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,emb_3,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
count,587.000000,582.000000,582.000000,582.000000,582.000000,582.000000,582.000000,582.000000,582.000000,582.000000,...,582.000000,582.000000,582.000000,582.000000,582.000000,582.000000,582.000000,582.000000,582.0,586.000000
mean,23.696500,0.190717,0.430999,0.378284,7.486254,-0.187567,0.001085,-0.020370,-0.010660,0.011101,...,-0.038588,0.012401,-0.029815,-0.019595,0.013994,-0.079011,0.014380,-0.005788,1.0,0.042662
std,2.138764,0.119790,0.152354,0.173783,3.920771,0.256688,0.020585,0.020501,0.021203,0.018494,...,0.019281,0.018312,0.020303,0.019072,0.022814,0.021981,0.020810,0.019486,0.0,0.995660
min,15.977302,0.013713,0.014903,0.009059,1.000000,-0.955694,-0.081310,-0.092300,-0.099959,-0.100358,...,-0.091741,-0.063716,-0.133065,-0.090775,-0.077995,-0.164592,-0.043651,-0.070156,1.0,-1.000000
25%,22.867464,0.093664,0.335205,0.251122,5.000000,-0.347712,-0.010367,-0.033006,-0.023011,-0.000481,...,-0.051330,0.001427,-0.043661,-0.032330,0.000187,-0.091882,0.000027,-0.018584,1.0,-1.000000
50%,23.883163,0.186457,0.432510,0.365186,7.000000,-0.178552,0.001891,-0.020780,-0.011008,0.011508,...,-0.038180,0.012262,-0.029508,-0.018805,0.012867,-0.078446,0.013227,-0.006544,1.0,1.000000
75%,24.726527,0.254087,0.528524,0.490210,10.000000,-0.010529,0.014027,-0.007697,0.000308,0.022425,...,-0.027040,0.024219,-0.017489,-0.008203,0.028533,-0.064615,0.026632,0.006664,1.0,1.000000
max,27.962915,0.953013,0.906664,0.970396,28.000000,0.933055,0.071203,0.039078,0.097658,0.059584,...,0.041117,0.085420,0.041488,0.080652,0.085982,-0.006161,0.103488,0.082590,1.0,1.000000


In [85]:
df.isna().sum()

Date                0
Price               0
avg_positive_XLF    5
avg_neutral_XLF     5
avg_negative_XLF    5
                   ..
emb_381             5
emb_382             5
emb_383             5
is_trading_day      5
Sign_next_day       1
Length: 393, dtype: int64

In [86]:
df.nunique()

Date                587
Price               516
avg_positive_XLF    582
avg_neutral_XLF     582
avg_negative_XLF    582
                   ... 
emb_381             582
emb_382             582
emb_383             582
is_trading_day        1
Sign_next_day         3
Length: 393, dtype: int64

### defining target - I want to assign:

class 1 to sign_next_day ==1

and class 0 to sign_next_day ∈ {0, -1}

In [87]:
df = df.dropna(subset=["Sign_next_day"])
df["Sign_next_day"] = (df["Sign_next_day"] == 1).astype(int)
df.head()

,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,-0.041143,0.014036,-0.033065,-0.033968,0.009230,-0.046950,0.004274,0.024519,1.0,0
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,-0.059143,0.007399,-0.055302,-0.039515,-0.000138,-0.066466,0.005075,-0.024416,1.0,0
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,-0.056243,0.016681,-0.040932,-0.007561,0.011109,-0.076687,0.010291,-0.010929,1.0,0
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,-0.033069,0.029601,-0.046962,-0.005147,0.038169,-0.097698,0.013953,-0.036602,1.0,1
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,-0.031487,0.017077,-0.037181,-0.028825,0.034319,-0.078715,0.038083,-0.010276,1.0,0


In [88]:
df.shape

(586, 393)

In [89]:
df.isna().sum()

Date                0
Price               0
avg_positive_XLF    5
avg_neutral_XLF     5
avg_negative_XLF    5
                   ..
emb_381             5
emb_382             5
emb_383             5
is_trading_day      5
Sign_next_day       0
Length: 393, dtype: int64

In [90]:
df.head()

,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,-0.041143,0.014036,-0.033065,-0.033968,0.009230,-0.046950,0.004274,0.024519,1.0,0
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,-0.059143,0.007399,-0.055302,-0.039515,-0.000138,-0.066466,0.005075,-0.024416,1.0,0
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,-0.056243,0.016681,-0.040932,-0.007561,0.011109,-0.076687,0.010291,-0.010929,1.0,0
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,-0.033069,0.029601,-0.046962,-0.005147,0.038169,-0.097698,0.013953,-0.036602,1.0,1
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,-0.031487,0.017077,-0.037181,-0.028825,0.034319,-0.078715,0.038083,-0.010276,1.0,0


In [91]:
df_catboost = df.copy()
df_catboost.head()

,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,-0.041143,0.014036,-0.033065,-0.033968,0.009230,-0.046950,0.004274,0.024519,1.0,0
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,-0.059143,0.007399,-0.055302,-0.039515,-0.000138,-0.066466,0.005075,-0.024416,1.0,0
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,-0.056243,0.016681,-0.040932,-0.007561,0.011109,-0.076687,0.010291,-0.010929,1.0,0
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,-0.033069,0.029601,-0.046962,-0.005147,0.038169,-0.097698,0.013953,-0.036602,1.0,1
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,-0.031487,0.017077,-0.037181,-0.028825,0.034319,-0.078715,0.038083,-0.010276,1.0,0


In [92]:
df_catboost["Date"] = pd.to_datetime(df_catboost["Date"]).dt.normalize()
df_catboost = df_catboost.sort_values("Date").reset_index(drop=True)

df_catboost["no_news"] = df_catboost["n_XLF"].isna().astype(int) # a day where there were no sector specific news
df_catboost.head(10)


,Date,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,2018-03-20,24.879965,0.061260,0.535400,0.403340,4.0,-0.342080,-0.003630,-0.025403,-0.005257,...,0.014036,-0.033065,-0.033968,0.009230,-0.046950,0.004274,0.024519,1.0,0,0
1,2018-03-21,24.871304,0.263512,0.485453,0.251035,14.0,0.012478,0.001563,-0.050504,-0.019049,...,0.007399,-0.055302,-0.039515,-0.000138,-0.066466,0.005075,-0.024416,1.0,0,0
2,2018-03-22,23.953358,0.281100,0.323045,0.395855,11.0,-0.114755,0.002994,-0.036866,-0.010311,...,0.016681,-0.040932,-0.007561,0.011109,-0.076687,0.010291,-0.010929,1.0,0,0
3,2018-03-23,23.225918,0.288174,0.307246,0.404580,6.0,-0.116405,-0.012174,-0.035777,-0.016196,...,0.029601,-0.046962,-0.005147,0.038169,-0.097698,0.013953,-0.036602,1.0,1,0
4,2018-03-26,23.979336,0.220507,0.469852,0.309642,11.0,-0.089135,-0.005624,-0.024752,-0.004313,...,0.017077,-0.037181,-0.028825,0.034319,-0.078715,0.038083,-0.010276,1.0,0,0
5,2018-03-27,23.511696,0.208302,0.501280,0.290419,8.0,-0.082117,0.008858,-0.044920,-0.005257,...,0.005357,-0.036538,-0.046846,0.031692,-0.089109,0.010091,0.004574,1.0,1,0
6,2018-03-28,23.554998,0.092945,0.652902,0.254153,8.0,-0.161208,0.003700,-0.017640,0.008789,...,0.029991,-0.031216,-0.029270,0.003462,-0.093493,-0.006516,0.024834,1.0,1,0
7,2018-03-29,23.875416,0.055242,0.592329,0.352428,5.0,-0.297186,0.000013,-0.013250,-0.006634,...,0.012979,-0.054554,-0.006435,-0.023151,-0.084078,0.012592,0.008806,1.0,0,0
8,2018-04-02,23.347151,0.046485,0.476633,0.476883,6.0,-0.430398,-0.013063,-0.034718,-0.003257,...,0.002121,-0.049837,0.019447,0.047689,-0.111716,0.016507,-0.010199,1.0,1,0
9,2018-04-03,23.667574,0.627429,0.324922,0.047649,3.0,0.579779,0.022106,-0.012358,0.013611,...,-0.010319,-0.062131,0.030860,0.067125,-0.091592,-0.007063,0.020872,1.0,1,0


In [93]:
df_catboost = df_catboost.drop(columns=["Date"])
df_catboost.head(1)

,Price,avg_positive_XLF,avg_neutral_XLF,avg_negative_XLF,n_XLF,sent_index_XLF,emb_0,emb_1,emb_2,emb_3,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,24.879965,0.06126,0.5354,0.40334,4.0,-0.34208,-0.00363,-0.025403,-0.005257,0.011237,...,0.014036,-0.033065,-0.033968,0.00923,-0.04695,0.004274,0.024519,1.0,0,0


In [94]:
original_features = [
    'Price',
    'avg_positive_XLF',
    'avg_neutral_XLF', 
    'avg_negative_XLF',
    'n_XLF',
    'sent_index_XLF',
    'is_trading_day'
]


embedding_features = [f'emb_{i}' for i in range(384)]

target = 'Sign_next_day'
df_clean = df_catboost.dropna(subset=[target, 'is_trading_day']).copy()


for emb_col in embedding_features:
    df_clean[emb_col] = df_clean[emb_col].fillna(0)


df_clean['avg_positive_XLF'] = df_clean['avg_positive_XLF'].fillna(0)
df_clean['avg_neutral_XLF'] = df_clean['avg_neutral_XLF'].fillna(1)
df_clean['avg_negative_XLF'] = df_clean['avg_negative_XLF'].fillna(0)
df_clean['n_XLF'] = df_clean['n_XLF'].fillna(0)
df_clean['sent_index_XLF'] = df_clean['sent_index_XLF'].fillna(0)

print(f"Clean dataset size: {len(df_clean)} rows")


Clean dataset size: 581 rows


In [95]:
X = df_clean[original_features + embedding_features]
y = df_clean[target]

#y = (y == 1).astype(int)

print(f"Target distribution: {y.value_counts().to_dict()}")

Target distribution: {1: 301, 0: 280}


In [96]:
embedding_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, random_state=12))  # 95% variance
])


original_pipeline = Pipeline([
    ('passthrough', FunctionTransformer())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('original', original_pipeline, original_features),
        ('embeddings', embedding_pipeline, embedding_features)
    ],
    remainder='drop'
)

In [97]:
catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    train_dir="cb_tmp",
    allow_writing_files=False
)

full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', catboost)
])


param_distributions = {
    'preprocessor__embeddings__pca__n_components': [0.90, 0.95, 0.98, 50, 75, 100],
    'classifier__depth': randint(3, 7),
    'classifier__learning_rate': uniform(0.01, 0.19),
    'classifier__iterations': randint(50, 500),
    'classifier__l2_leaf_reg': uniform(1, 14),
    'classifier__border_count': randint(32, 128),
    'classifier__random_strength': uniform(0.5, 5),
    'classifier__min_data_in_leaf': randint(5, 30),
    'classifier__rsm': uniform(0.6, 0.4),
    'classifier__boosting_type': ['Ordered', 'Plain'],
    'classifier__bootstrap_type': ['Bernoulli', 'MVS'],
    'classifier__subsample': uniform(0.6, 0.4),
}

random_search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_distributions,
    n_iter=40,
    cv=5,
    scoring='roc_auc',
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)


random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print(f"Best parameters:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")



Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.6007
Best parameters:
  classifier__boosting_type: Plain
  classifier__bootstrap_type: MVS
  classifier__border_count: 41
  classifier__depth: 4
  classifier__iterations: 175
  classifier__l2_leaf_reg: 14.894104741865496
  classifier__learning_rate: 0.0184927614253773
  classifier__min_data_in_leaf: 24
  classifier__random_strength: 1.5100847636008097
  classifier__rsm: 0.9225176102675893
  classifier__subsample: 0.6462402143198337
  preprocessor__embeddings__pca__n_components: 75


In [98]:
best_preprocessor = random_search.best_estimator_.named_steps['preprocessor']
pca = best_preprocessor.named_transformers_['embeddings'].named_steps['pca']

print(f"Number of prncipals components kept: {pca.n_components_}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.4f}")
print(f"Original embedding dimensions (384) were reduced to {pca.n_components_}")


Number of prncipals components kept: 75
Explained variance ratio: 0.7750
Original embedding dimensions (384) were reduced to 75
